In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import torch

from diffdrr.drr import DRR
from diffdrr.data import load_example_ct
from diffdrr.visualization import plot_drr

In [ ]:
# Read in the volume
volume, spacing = load_example_ct()
device = "cuda" if torch.cuda.is_available() else "cpu"

# Get parameters for the detector
bx, by, bz = np.array(volume.shape) * np.array(spacing) / 2
translations = torch.tensor([[bx, by, bz]]).to(device)
rotations = torch.tensor([[np.pi, 0, np.pi / 2]]).to(device)

In [ ]:
#|cuda
height = 100

drr = DRR(volume, spacing, sdr=300.0, height=height, delx=4.0).to(device)
%timeit drr(rotations, translations)
del drr

10.7 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#|cuda
height = 200

drr = DRR(volume, spacing, sdr=300.0, height=height, delx=4.0).to("cuda" if torch.cuda.is_available() else "cpu")
%timeit drr(rotations, translations)
del drr

35.2 ms ± 34.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#|cuda
height = 300

drr = DRR(volume, spacing, sdr=300.0, height=height, delx=4.0).to("cuda" if torch.cuda.is_available() else "cpu")
%timeit drr(rotations, translations)
del drr

74.7 ms ± 57.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#|cuda
height = 400

drr = DRR(volume, spacing, sdr=300.0, height=height, delx=4.0).to("cuda" if torch.cuda.is_available() else "cpu")
%timeit drr(rotations, translations)
del drr

126 ms ± 80.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#|cuda
height = 500

drr = DRR(volume, spacing, sdr=300.0, height=height, delx=4.0).to("cuda" if torch.cuda.is_available() else "cpu")
%timeit drr(rotations, translations)
del drr

186 ms ± 29.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Memory constraints

Up until this point, we could compute every ray in the DRR in one go on the GPU. However, as the DRRs get bigger, we will quickly run out of memory. For example, on a 12 GB GPU, computing a 600 by 600 DRR will raise a CUDA memory error.

Instead, we can compute patches of the DRR at a time. Pass `patch_size` to the `DRR` module to specify the size of the patch. Note the patch size must evenly tile (`height`, `width`).

In [ ]:
#|cuda
height = 600
patch_size = 150

drr = DRR(volume, spacing, sdr=300.0, height=height, delx=4.0, patch_size=patch_size).to("cuda" if torch.cuda.is_available() else "cpu")
%timeit drr(rotations, translations)
del drr

189 ms ± 56.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#|cuda
height = 750
patch_size = 150

drr = DRR(volume, spacing, sdr=300.0, height=height, delx=4.0, patch_size=patch_size).to("cuda" if torch.cuda.is_available() else "cpu")
%timeit drr(rotations, translations)
del drr

270 ms ± 224 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#|cuda
height = 1000
patch_size = 250

drr = DRR(volume, spacing, sdr=300.0, height=height, delx=4.0, patch_size=patch_size).to("cuda" if torch.cuda.is_available() else "cpu")
%timeit drr(rotations, translations)
del drr

429 ms ± 235 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#|cuda
height = 1500
patch_size = 250

drr = DRR(volume, spacing, sdr=300.0, height=height, delx=4.0, patch_size=patch_size).to("cuda" if torch.cuda.is_available() else "cpu")
%timeit drr(rotations, translations)
del drr

854 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


With `patch_size`, the only limitation is DRR storage, not computation.